In [1]:
pip install spotipy 

     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 153kB 29.6MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="a006ea8174bc4689b4eb39c47b5449a1" 
secret = "1cca3d1fff6145fdaee72ba822e8b586" 

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
import timeit
import pandas as pd
import torch

In [4]:
def getTracks(query, number):
  start = timeit.default_timer()

  # create empty lists where the results are going to be stored
  artist_name = []
  track_name = []
  popularity = []
  track_id = []

  for i in range(0,number,10):
      track_results = sp.search(q=query, type='track', limit=10,offset=i)
      for i, t in enumerate(track_results['tracks']['items']):
          artist_name.append(t['artists'][0]['name'])
          track_name.append(t['name'])
          track_id.append(t['id'])
          popularity.append(t['popularity'])
        

  stop = timeit.default_timer()
  print ('Time to run this code (in seconds):', stop - start)
  df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
  return df_tracks

In [5]:
def cleanDuplicates(tracks):
  grouped = tracks.groupby(['artist_name','track_name'], as_index=True).size()
  grouped[grouped > 1].count()
  tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)
  

In [6]:
def getAudioFeatures(tracks):
  # again measuring the time
  start = timeit.default_timer()

  # empty list, batchsize and the counter for None results
  rows = []
  batchsize = 100
  None_counter = 0

  for i in range(0,len(tracks['track_id']),batchsize):
      batch = tracks['track_id'][i:i+batchsize]
      feature_results = sp.audio_features(batch)
      for i, t in enumerate(feature_results):
          if t == None:
              None_counter = None_counter + 1
          else:
              rows.append(t)
              
  print('Number of tracks where no audio features were available:',None_counter)

  stop = timeit.default_timer()
  print ('Time to run this code (in seconds):',stop - start)
  df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
  return df_audio_features

In [7]:
def dropColumns(df_audio_features):
  columns_to_drop = ['analysis_url','track_href','type','key','mode','time_signature','uri']
  df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

  df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

  df_audio_features.shape

In [8]:
def mergeDataframes(df_tracks,df_audio_features):
  # merge both dataframes
  # the 'inner' method will make sure that we only keep track IDs present in both datasets
  df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
  
  return df

In [10]:
df = pd.read_csv('data_moods.csv')

In [11]:
df

,name,album,artist,id,release_date,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature,mood
0,1999,1999,Prince,2H7PHVdQ3mXqEHXcvclTB0,1982-10-27,68,379266,0.866,0.13700,0.7300,0.000000,0.0843,0.6250,-8.201,0.0767,118.523,5,4,Happy
1,23,23,Blonde Redhead,4HIwL9ii9CcXpTOTzMq0MP,2007-04-16,43,318800,0.381,0.01890,0.8320,0.196000,0.1530,0.1660,-5.069,0.0492,120.255,8,4,Sad
2,9 Crimes,9,Damien Rice,5GZEeowhvSieFDiR8fQ2im,2006-11-06,60,217946,0.346,0.91300,0.1390,0.000077,0.0934,0.1160,-15.326,0.0321,136.168,0,4,Sad
3,99 Luftballons,99 Luftballons,Nena,6HA97v4wEGQ5TUClRM0XLc,1984-08-21,2,233000,0.466,0.08900,0.4380,0.000006,0.1130,0.5870,-12.858,0.0608,193.100,4,4,Happy
4,A Boy Brushed Red Living In Black And White,They're Only Chasing Safety,Underoath,47IWLfIKOKhFnz1FUEUIkE,2004-01-01,60,268000,0.419,0.00171,0.9320,0.000000,0.1370,0.4450,-3.604,0.1060,169.881,1,4,Energetic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,windcatcher,windcatcher,Leo Nocta,59VApBbrS2IADQk4ml5mdo,2020-06-19,36,123066,0.402,0.96100,0.2360,0.919000,0.0921,0.1460,-20.615,0.0603,129.736,0,3,Calm
682,yellow is the color of her eyes,yellow is the color of her eyes,Soccer Mommy,4D3nttJPU6L0M2epr7sId6,2019-11-19,5,435080,0.452,0.75700,0.5150,0.120000,0.1400,0.1910,-7.351,0.0255,80.537,11,4,Sad
683,you broke me first,you broke me first,Tate McRae,45bE4HXI0AwGZXfZtMp8JR,2020-04-17,87,169265,0.642,0.78600,0.3740,0.000000,0.0906,0.0799,-9.386,0.0545,124.099,4,4,Sad
684,you were good to me,brent,Jeremy Zucker,4CxFN5zON70B3VOPBYbd6P,2019-05-03,76,219146,0.561,0.91300,0.0848,0.000026,0.1120,0.2060,-15.099,0.0404,102.128,2,4,Sad


In [12]:
df = df.sample(frac=1).reset_index(drop=True)

In [13]:
df

,name,album,artist,id,release_date,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature,mood
0,Litost,Litost,Minxz,0bMcjygjhkTVB8T4r3xCde,2019-04-26,22,152456,0.383,0.98800,0.101,0.919000,0.1020,0.242,-18.306,0.0427,119.709,0,3,Calm
1,The Hunger,The Hunger,Bat For Lashes,7AaSbMyCeUxcGsvCmzmfEJ,2019-08-05,26,299000,0.436,0.04020,0.753,0.020400,0.1480,0.399,-9.760,0.0359,89.968,4,4,Sad
2,As I Sleep (feat. Charlee),As I Sleep (feat. Charlee),Tobtok,72qlZsoVPpyFxVjL6GUuYI,2018-04-13,55,193471,0.694,0.01950,0.824,0.042600,0.3270,0.202,-6.306,0.1610,123.014,8,4,Energetic
3,Children Of Today,Children Of Today,Blasterjaxx,2106G7MY2XnVyqij2bJVGH,2019-04-19,58,169384,0.507,0.00892,0.836,0.000598,0.4850,0.333,-4.357,0.0380,130.027,6,4,Energetic
4,Bad Feeling,Bad Feeling,Muzz,1qUaWLUot3Iik95S09cdSZ,2020-03-12,39,157930,0.512,0.35100,0.352,0.740000,0.1060,0.275,-9.093,0.0354,82.943,5,4,Sad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,ID8,ID8,Dimitri Vangelis & Wyman,2jxfHzOYVLmGKejNRHvqoc,2020-03-06,51,197183,0.576,0.01980,0.948,0.000123,0.6830,0.193,-4.251,0.0898,128.029,9,3,Happy
682,It's Goin' Down (feat. Mike Shinoda & Mr. Hahn),Built From Scratch,X-Ecutioners,6Xb5klX1nnSUxoh2AyZd54,2002-02-26,40,248573,0.674,0.00165,0.890,0.013600,0.0465,0.900,-3.953,0.2740,104.862,8,4,Energetic
683,Surrender,Surrender,Natalie Taylor,0ecC8p17cDNlxHXkuYqeR6,2019-10-29,84,183103,0.390,0.80000,0.448,0.182000,0.1070,0.077,-8.570,0.0276,144.890,5,5,Sad
684,The Temporary,The Temporary,Kate Teague,5g5dwJD6zwXG81LRdEsnQr,2020-06-16,36,219018,0.549,0.29200,0.524,0.036600,0.1220,0.229,-8.288,0.0261,94.991,0,4,Sad


In [14]:
#Libraries to pre-process the variables
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split

In [15]:
col_features = df.columns[7:16]
features = df[col_features]
mood = df['mood']
features = MinMaxScaler().fit_transform(features)
#Encode the labels (targets)
encoder = LabelEncoder()
encoder.fit(mood)
encoded_y = encoder.transform(mood)
#Split train and test data with a test size of 20%
features_train,features_test,mood_train,mood_test = train_test_split(features,encoded_y,test_size=0.2,random_state=15)

In [16]:
features.shape


(686, 9)

In [17]:
print(features)

[[0.3527433  0.99196783 0.10044222 ... 0.54686347 0.04964358 0.41169531]
 [0.41422109 0.04035628 0.75723021 ... 0.74395756 0.03233198 0.23359483]
 [0.71349031 0.01957304 0.8287516  ... 0.82361624 0.35081466 0.43148692]
 ...
 [0.36086301 0.80321179 0.44999043 ... 0.77140221 0.01120163 0.56248877]
 [0.54529637 0.29316889 0.52654854 ... 0.7779059  0.00738289 0.26367447]
 [0.47685883 0.09377022 0.52352651 ... 0.74762454 0.0160387  0.34024792]]


In [18]:
len(features_train)

548

In [19]:
len(features_test)

138

In [20]:
len(mood_train)

548

In [21]:
len(mood_test)

138

In [22]:
#Libraries to create the Multi-class Neural Network
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
#Import tensorflow and disable the v2 behavior and eager mode
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.disable_v2_behavior()

In [23]:
#Function that creates the structure of the Neural Network
def base_model():
    #Create the model
   model = Sequential()
#Add 1 layer with 8 nodes,input of 4 dim with relu function
   model.add(Dense(8,input_dim=9,activation='relu'))
#Add 1 layer with output 3 and softmax function
   model.add(Dense(4,activation='softmax'))
#Compile the model using logistic loss function and adam     optimizer, accuracy correspond to the metric displayed
   model.compile(loss='categorical_crossentropy',optimizer='adam',
              metrics=['accuracy'])
   return model
#Configure the estimator with 300 epochs and 200 batchs. the build_fn takes the function defined above.
estimator = KerasClassifier(build_fn=base_model,epochs=300,batch_size=64)

In [24]:
#Library to evaluate the model
from sklearn.model_selection import cross_val_score, KFold

In [25]:
#Evaluate the model using KFold cross validation
kfold = KFold(n_splits=10,shuffle=True)
results = cross_val_score(estimator,features,encoded_y,cv=kfold)
print("%.2f%% (%.2f%%)" % (results.mean()*100,results.std()*100))

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch 209/300
10/10 [==============================] - 0s 3ms/step - loss: 0.4798 - accuracy: 0.8083
Epoch 210/300
10/10 [==============================] - 0s 2ms/step - loss: 0.4426 - accuracy: 0.8253
Epoch 211/300
10/10 [==============================] - 0s 2ms/step - loss: 0.4719 - accuracy: 0.8128
Epoch 212/300
10/10 [==============================] - 0s 3ms/step - loss: 0.4353 - accuracy: 0.8087
Epoch 213/300
10/10 [==============================] - 0s 3ms/step - loss: 0.4383 - accuracy: 0.8253
Epoch 214/300
10/10 [==============================] - 0s 3ms/step - loss: 0.4803 - accuracy: 0.8044
Epoch 215/300
10/10 [==============================] - 0s 2ms/step - loss: 0.4756 - accuracy: 0.8034
Epoch 216/300
10/10 [==============================] - 0s 3ms/step - loss: 0.4350 - accuracy: 0.8284
Epoch 217/300
10/10 [==============================] - 0s 2ms/step - loss: 0.4509 - accuracy: 0.8146
Epoch 218/300
10/10 [==============================] - 0s 2ms/st

In [26]:
#Train the model with the train data
estimator.fit(features_train,mood_train)
#Predict the model with the test data
mood_preds = estimator.predict(features_test)

Epoch 1/300
9/9 [==============================] - 1s 2ms/step - loss: 1.4985 - accuracy: 0.1969
Epoch 2/300
9/9 [==============================] - 0s 3ms/step - loss: 1.4402 - accuracy: 0.2225
Epoch 3/300
9/9 [==============================] - 0s 3ms/step - loss: 1.4472 - accuracy: 0.1933
Epoch 4/300
9/9 [==============================] - 0s 4ms/step - loss: 1.4194 - accuracy: 0.2265
Epoch 5/300
9/9 [==============================] - 0s 3ms/step - loss: 1.3911 - accuracy: 0.2442
Epoch 6/300
9/9 [==============================] - 0s 3ms/step - loss: 1.3695 - accuracy: 0.3121
Epoch 7/300
9/9 [==============================] - 0s 3ms/step - loss: 1.3396 - accuracy: 0.3384
Epoch 8/300
9/9 [==============================] - 0s 2ms/step - loss: 1.3130 - accuracy: 0.4606
Epoch 9/300
9/9 [==============================] - 0s 3ms/step - loss: 1.2956 - accuracy: 0.5050
Epoch 10/300
9/9 [==============================] - 0s 2ms/step - loss: 1.2605 - accuracy: 0.5257
Epoch 11/300
9/9 [===========

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [27]:
import random

In [28]:

def getRandomSearch():
  # A list of all characters that can be chosen.
  characters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
  
  # Gets a random character from the characters string.
  randomCharacter = characters[random.randint(0,25)]
  randomSearch = '';

  #Places the wildcard character at the beginning, or both beginning and end, randomly.
  switcher = random.randint(0,1)
  if (switcher == 0):
    randomSearch = randomCharacter + '%'
  else:
    randomSearch = '%' + randomCharacter + '%'

  

  return randomSearch;


In [29]:
test = getTracks(getRandomSearch(),50)

Time to run this code (in seconds): 0.6019446290000019


In [30]:
test

,artist_name,track_name,track_id,popularity
0,Post Malone,Circles,21jGcNKet2qwijlDFuPiPb,90
1,Lil Nas X,MONTERO (Call Me By Your Name),3BZEcbdtXQSo7OrvKRJ6mb,97
2,Juice WRLD,Come & Go (with Marshmello),2Y0wPrPQBrGhoLn14xRYCG,86
3,Doja Cat,Streets,60ynsPSSKe6O3sfwRnIBRf,95
4,Megan Thee Stallion,Cry Baby (feat. DaBaby),4aarlAfLKVCTxUDNgbwhjH,84
5,Cardi B,Up,1XXimziG1uhM0eDNCZCrUl,93
6,Gym Class Heroes,Cupid's Chokehold / Breakfast in America,2Lhdl74nwwVGOE2Gv35QuK,87
7,Beach Bunny,Cloud 9,6vFsBXYczYsP0H3lgunZOm,85
8,Drake,Laugh Now Cry Later (feat. Lil Durk),2SAqBLGA283SUiwJ3xOUVI,89
9,Morgan Wallen,Chasin' You,5MwynWK9s4hlyKHqhkNn4A,80


In [31]:
test_features = getAudioFeatures(test)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 0.11277386299997261


In [32]:
dropColumns(test_features)

In [33]:
df_test = mergeDataframes(test,test_features)

In [34]:
col_features = df_test.columns[4:13]
features = df[col_features]
features = MinMaxScaler().fit_transform(features)

In [35]:
mood_preds_test = estimator.predict(features)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [36]:
mood_preds_test

array([2, 3, 3, 3, 2, 2, 3, 2, 2, 2, 3, 2, 3, 2, 3, 2, 3, 3, 3, 2, 2, 3,
       3, 2, 2, 3, 3, 3, 2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 3, 3, 2, 2, 2,
       3, 2, 2, 2, 3, 3, 3, 2, 3, 3, 2, 2, 3, 2, 2, 2, 3, 2, 3, 3, 0, 2,
       3, 1, 3, 2, 2, 2, 3, 2, 3, 3, 3, 2, 3, 2, 2, 3, 2, 2, 2, 2, 3, 3,
       2, 3, 3, 1, 3, 2, 2, 3, 2, 3, 3, 2, 3, 3, 2, 2, 2, 3, 3, 3, 2, 2,
       3, 3, 2, 1, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 3, 2, 1,
       2, 2, 3, 3, 3, 2, 2, 3, 3, 2, 3, 3, 2, 3, 3, 2, 3, 1, 2, 2, 2, 2,
       2, 0, 3, 3, 2, 3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 3, 2, 2, 3, 3,
       3, 2, 3, 3, 2, 2, 3, 3, 3, 1, 3, 2, 2, 3, 2, 2, 2, 3, 3, 2, 2, 3,
       2, 2, 3, 2, 2, 2, 2, 1, 3, 3, 3, 2, 2, 2, 3, 2, 2, 2, 3, 3, 3, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 2, 3, 2, 2, 2, 1, 3, 3,
       3, 2, 3, 2, 3, 0, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 2, 2, 3, 2, 3, 2,
       2, 2, 2, 3, 2, 2, 1, 3, 3, 2, 2, 2, 3, 2, 2, 3, 3, 3, 1, 3, 3, 2,
       3, 3, 2, 2, 2, 3, 2, 3, 3, 2, 3, 2, 2, 2, 3,

In [37]:
IDs = test['track_id']
names = test['track_name']

In [38]:
results = []

for x in range(50):
  if (mood_preds_test[x] == 2):
    results.append([names[x],IDs[x]])

In [39]:
results

[['Circles', '21jGcNKet2qwijlDFuPiPb'],
 ['Cry Baby (feat. DaBaby)', '4aarlAfLKVCTxUDNgbwhjH'],
 ['Up', '1XXimziG1uhM0eDNCZCrUl'],
 ['Cloud 9', '6vFsBXYczYsP0H3lgunZOm'],
 ['Laugh Now Cry Later (feat. Lil Durk)', '2SAqBLGA283SUiwJ3xOUVI'],
 ["Chasin' You", '5MwynWK9s4hlyKHqhkNn4A'],
 ['Classic', '6FE2iI43OZnszFLuLtvvmg'],
 ['Chanel', '6Nle9hKrkL1wQpwNfEkxjh'],
 ['Congratulations', '3a1lNhkSLSkpJE4MSHpDu9'],
 ['Can I Call You Tonight?', '64lsIF5pw0sJY0gV5kz0RN'],
 ['Beautiful Crazy', '2rxQMGVafnNaRaXlRMWPde'],
 ['Closer', '7BKLCZ1jbUBVqRi2FVlTVw'],
 ['WITHOUT YOU', '27OeeYzk6klgBh83TSvGMA'],
 ['champagne problems', '0sY6ZUTh4yoctD8VIXz339'],
 ['WAP (feat. Megan Thee Stallion)', '4Oun2ylbjFKMPTiaSbbCih'],
 ['Can You Feel My Heart', '1PInWkBARsxLyouDqa2GtF'],
 ['Are You Bored Yet? (feat. Clairo)', '57RA3JGafJm5zRtKJiKPIm'],
 ["I Can't Handle Change", '0I1eFRytp4XRhLCjT6tZm7'],
 ['DAYWALKER! (feat. CORPSE)', '2NnJpRXIlx35Vij3bPZO0h'],
 ['CLOUDS', '5UMMPHPp6vRP6ghPpSUOzp'],
 ['Whoopty', '5v

In [40]:
for i in range(len(results)):
  result = sp.track(results[i][1])
  
  print('track    : ' + result['name'])
  print('audio    : ' + str(result['external_urls']))
  print('cover art: ' + result['album']['images'][0]['url'])
  print()

track    : Circles
audio    : {'spotify': 'https://open.spotify.com/track/21jGcNKet2qwijlDFuPiPb'}
cover art: https://i.scdn.co/image/ab67616d0000b2739478c87599550dd73bfa7e02

track    : Cry Baby (feat. DaBaby)
audio    : {'spotify': 'https://open.spotify.com/track/4aarlAfLKVCTxUDNgbwhjH'}
cover art: https://i.scdn.co/image/ab67616d0000b273da256972582b455d46985ba9

track    : Up
audio    : {'spotify': 'https://open.spotify.com/track/1XXimziG1uhM0eDNCZCrUl'}
cover art: https://i.scdn.co/image/ab67616d0000b273d619b8baab0619516bb53804

track    : Cloud 9
audio    : {'spotify': 'https://open.spotify.com/track/6vFsBXYczYsP0H3lgunZOm'}
cover art: https://i.scdn.co/image/ab67616d0000b273d6fd719531afda5f9cc0e248

track    : Laugh Now Cry Later (feat. Lil Durk)
audio    : {'spotify': 'https://open.spotify.com/track/2SAqBLGA283SUiwJ3xOUVI'}
cover art: https://i.scdn.co/image/ab67616d0000b27352c75ed37313b889447011ef

track    : Chasin' You
audio    : {'spotify': 'https://open.spotify.com/track/5M